# Langchain + Pinecone
You will learn the following things:

* Create embeddings from a document
* Save these embeddings to Pinecone indexes
* Query these Pinecone 

There is a `requirements.txt` file, install all the dependencies from it into a new virtual environment.

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Pinecone
from PyPDF2 import PdfReader
import pinecone

from config import config

* Reading a PDF file from the path
* Creating and saving the content of the PDF file into a `output.txt` file

In [ ]:
file_path = 'data/inputs/sample.pdf'

In [ ]:
reader = PdfReader(file_path)
text = ''
for page in reader.pages:
    text += page.extract_text()

with open(f'data/outputs/output.txt', 'w') as file:
    file.write(text)

* Loading the folder with `.txt` files

In [ ]:
loader = DirectoryLoader(
    'data/outputs/',
    glob='**/*.txt',
    loader_cls=TextLoader
)

In [ ]:
documents = loader.load()

* Split the documents into chonk of text

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=0
)

In [ ]:
texts = text_splitter.split_documents(documents)

In [ ]:
embeddings = OpenAIEmbeddings(
    openai_api_key=config.OPENAI_API_KEY
)

* Initialize `Pinecone`

In [ ]:
pinecone.init(
    api_key=config.PINECONE_API_KEY,
    environment=config.PINECONE_ENVIRONMENT,
)

In [ ]:
index_name = 'test'

* Create indexes in Pinecone

In [ ]:
vectordb = Pinecone.from_documents(
    documents=texts,
    embedding=embeddings,
    index_name=index_name
)

* Load the existing indexes

In [ ]:
db = Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

* QnA chain without `memory`

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0.0),
    chain_type="stuff",
    retriever=db.as_retriever()
)

In [ ]:
result = qa.run('what are the different spiritual laws?')

In [ ]:
print(result)

* QnA with memory

In [ ]:
cqa = ConversationalRetrievalChain.from_llm(
    llm=ChatOpenAI(temperature=0.0),
    retriever=db.as_retriever()
)

In [ ]:
chat_history = []
query = 'what are the different spiritual laws?'
result = cqa({'question': query, 'chat_history': chat_history})

In [ ]:
result

In [ ]:
chat_history.append((query, result['answer']))

In [ ]:
query = 'can you explain the first one?'
result = cqa({'question': query, 'chat_history': chat_history})

In [ ]:
result